### Repositorio
https://github.com/Diego2436/Inteligencia-Artificial/
### Nombres
- Olmos Verdin Diego
- Romero Hernández Oscar David
- Valentin Ramos Emmanuel Guadalupe

In [3]:
import numpy as np
from sklearn.datasets import load_iris, load_wine, load_breast_cancer

Importa la biblioteca NumPy para realizar operaciones matemáticas y de álgebra lineal de manera eficiente.
Importa el módulo datasets de la biblioteca scikit-learn, que es una biblioteca de machine learning en Python.

load_iris: carga el conjunto de datos Iris, utilizado comúnmente en clasificación.
load_wine: carga el conjunto de datos Wine, utilizado también en problemas de clasificación.
load_breast_cancer: carga el conjunto de datos Breast Cancer, útil para problemas de clasificación binaria.

## Paso 1: Cargar los datasets

In [4]:
datasets = {
    "Iris": load_iris(),
    "Wine": load_wine(),
    "Breast Cancer": load_breast_cancer()
}

# Separar los datos y etiquetas de cada dataset
dataset_data = {name: (data.data, data.target) for name, data in datasets.items()}

Define un diccionario llamado datasets que almacena tres conjuntos de datos de ejemplo cargados con funciones de scikit-learn: load_iris, load_wine y load_breast_cancer. 
Las claves del diccionario son los nombres de los conjuntos de datos ("Iris", "Wine", y "Breast Cancer"), y los valores son los conjuntos de datos correspondientes.
Crea un diccionario dataset_data que organiza y separa los datos y etiquetas (objetivo) de cada conjunto de datos. Usa una comprensión de diccionario para recorrer cada elemento en datasets. Para cada conjunto de datos:

name representa el nombre del conjunto de datos.
data.data contiene las características del conjunto de datos.
data.target contiene las etiquetas de clasificación.
Así, dataset_data almacenará una tupla (data, target) para cada conjunto de datos bajo su respectivo nombre.

## Paso 2: Implementación de clasificadores

In [5]:
def euclidean_classifier(X_train, y_train, X_test):
    # Calcular el centroide (media) de cada clase en el conjunto de entrenamiento
    classes = np.unique(y_train)
    centroids = {cls: X_train[y_train == cls].mean(axis=0) for cls in classes}
    
    # Asignar cada muestra de X_test a la clase con el centroide más cercano
    y_pred = []
    for x in X_test:
        distances = {cls: np.linalg.norm(x - centroid) for cls, centroid in centroids.items()}
        y_pred.append(min(distances, key=distances.get))
    return np.array(y_pred)

def knn_1_classifier(X_train, y_train, X_test):
    y_pred = []
    for x in X_test:
        distances = np.linalg.norm(X_train - x, axis=1)
        nearest_neighbor = np.argmin(distances)
        y_pred.append(y_train[nearest_neighbor])
    return np.array(y_pred)

La función `euclidean_classifier` clasifica muestras basándose en la distancia euclidiana entre cada muestra y el centroide (media) de cada clase en el conjunto de entrenamiento `X_train`. Primero, obtiene las clases únicas de `y_train` y calcula el centroide de cada clase usando una comprensión de diccionario. Luego, para cada muestra en `X_test`, calcula la distancia a cada centroide y asigna la clase correspondiente al centroide más cercano, devolviendo las predicciones como un array `y_pred`. Por otro lado, la función `knn_1_classifier` implementa un clasificador k-NN con `k=1`, donde cada muestra de prueba en `X_test` se clasifica según la clase de su vecino más cercano en `X_train`. Para cada muestra en `X_test`, calcula las distancias euclidianas a todas las muestras de `X_train`, identifica el índice de la muestra más cercana y asigna la clase correspondiente a esa muestra en `y_train`. Finalmente, devuelve un array `y_pred` con las clases predichas para cada muestra en `X_test`. Ambas funciones, basadas en distancias euclidianas, permiten clasificar puntos de prueba usando diferentes enfoques: el primero con centroides de clases y el segundo con vecinos individuales del conjunto de entrenamiento.

## Paso 3: Implementación de métodos de validación

In [6]:
# Modificación en hold_out_validation para incluir el número de clases
def hold_out_validation(X, y, classifier_func, test_size=0.3, random_seed=42):
    # Número total de clases en el conjunto completo
    num_classes = len(np.unique(y))
    
    # Dividir manualmente en entrenamiento y prueba
    np.random.seed(random_seed)
    indices = np.random.permutation(len(X))
    split_point = int(len(X) * (1 - test_size))
    train_idx, test_idx = indices[:split_point], indices[split_point:]
    
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    y_pred = classifier_func(X_train, y_train, X_test)
    accuracy = np.mean(y_pred == y_test)
    # Pasar num_classes a la función de matriz de confusión
    conf_matrix = calculate_confusion_matrix(y_test, y_pred, num_classes)
    return accuracy, conf_matrix

# Modificación en cross_validation_kfold para incluir el número de clases
def cross_validation_kfold(X, y, classifier_func, k=10):
    n_samples = len(X)
    indices = np.arange(n_samples)
    np.random.shuffle(indices)
    fold_size = n_samples // k
    num_classes = len(np.unique(y))  # Número total de clases en el conjunto completo
    
    accuracies = []
    conf_matrix_sum = np.zeros((num_classes, num_classes), dtype=int)
    
    for i in range(k):
        test_idx = indices[i * fold_size:(i + 1) * fold_size]
        train_idx = np.setdiff1d(indices, test_idx)
        
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        
        y_pred = classifier_func(X_train, y_train, X_test)
        accuracies.append(np.mean(y_pred == y_test))
        # Pasar num_classes a la función de matriz de confusión
        conf_matrix_sum += calculate_confusion_matrix(y_test, y_pred, num_classes)
    
    accuracy = np.mean(accuracies)
    return accuracy, conf_matrix_sum

# Función de matriz de confusión que recibe el número total de clases
def calculate_confusion_matrix(y_true, y_pred, num_classes):
    # Crear matriz con el tamaño basado en el total de clases del dataset completo
    matrix = np.zeros((num_classes, num_classes), dtype=int)
    for true_label, pred_label in zip(y_true, y_pred):
        matrix[true_label, pred_label] += 1
    return matrix

# Modificación en leave_one_out_validation para incluir el número de clases
def leave_one_out_validation(X, y, classifier_func):
    n_samples = len(X)
    accuracies = []
    num_classes = len(np.unique(y))  # Número total de clases en el conjunto completo
    conf_matrix_sum = np.zeros((num_classes, num_classes), dtype=int)
    
    for i in range(n_samples):
        X_train = np.concatenate((X[:i], X[i+1:]))
        y_train = np.concatenate((y[:i], y[i+1:]))
        X_test = X[i:i+1]
        y_test = y[i:i+1]
        
        y_pred = classifier_func(X_train, y_train, X_test)
        accuracies.append(y_pred[0] == y_test[0])
        
        # Pasar el número de clases a la función de matriz de confusión
        conf_matrix_sum += calculate_confusion_matrix(y_test, y_pred, num_classes)
    
    accuracy = np.mean(accuracies)
    return accuracy, conf_matrix_sum

La función `hold_out_validation` divide el conjunto de datos en entrenamiento y prueba utilizando un porcentaje de división (`test_size`) y una semilla aleatoria (`random_seed`). Calcula el número total de clases en `y`, lo cual pasa a la función `calculate_confusion_matrix`, para construir la matriz de confusión de tamaño adecuado. Luego, entrena el clasificador (`classifier_func`) con el conjunto de entrenamiento, predice sobre el conjunto de prueba y calcula la exactitud (`accuracy`). La función `cross_validation_kfold` implementa validación cruzada k-fold dividiendo aleatoriamente `X` y `y` en `k` pliegues. Para cada pliegue, se usa como conjunto de prueba y los restantes como conjunto de entrenamiento; los resultados de las predicciones se acumulan en una matriz de confusión (`conf_matrix_sum`) y en una lista de exactitudes, promediando al final para obtener una exactitud general. La función `leave_one_out_validation` aplica validación cruzada de "uno contra todos" (`leave-one-out`), donde cada muestra individual se usa sucesivamente como conjunto de prueba y el resto como conjunto de entrenamiento. Cada predicción se evalúa en términos de exactitud y se acumula en `conf_matrix_sum`. Finalmente, la función `calculate_confusion_matrix` genera la matriz de confusión utilizando `y_true` y `y_pred` y el total de clases (`num_classes`), creando una matriz cuadrada que cuenta la frecuencia de cada combinación de etiquetas verdaderas y predichas. Estas funciones de validación han sido modificadas para considerar el número de clases en el conjunto de datos, garantizando que las matrices de confusión resultantes incluyan todas las clases posibles, aun cuando alguna clase no aparezca en un conjunto de prueba específico.

## Paso 4: Ejecutar la validación y calcular el desempeño 

In [7]:
for dataset_name, (X, y) in dataset_data.items():
    print(f"Dataset: {dataset_name}")
    for classifier_name, classifier_func in [("Euclidean", euclidean_classifier), ("1-NN", knn_1_classifier)]:
        print(f"\nClasificador: {classifier_name}")
        
        # Hold-Out
        accuracy, conf_matrix = hold_out_validation(X, y, classifier_func)
        print(f"Hold Out (70/30) - Accuracy: {accuracy:.4f}")
        print("Matriz de Confusión:\n", conf_matrix)
        
        # 10-Fold Cross-Validation
        accuracy, conf_matrix = cross_validation_kfold(X, y, classifier_func)
        print(f"10-Fold Cross-Validation - Accuracy: {accuracy:.4f}")
        print("Matriz de Confusión:\n", conf_matrix)
        
        # Leave-One-Out
        accuracy, conf_matrix = leave_one_out_validation(X, y, classifier_func)
        print(f"Leave-One-Out - Accuracy: {accuracy:.4f}")
        print("Matriz de Confusión:\n", conf_matrix)
    print("\n" + "-"*40 + "\n")

Dataset: Iris

Clasificador: Euclidean
Hold Out (70/30) - Accuracy: 0.8889
Matriz de Confusión:
 [[10  0  0]
 [ 0 15  2]
 [ 0  3 15]]
10-Fold Cross-Validation - Accuracy: 0.9267
Matriz de Confusión:
 [[50  0  0]
 [ 0 45  5]
 [ 0  6 44]]
Leave-One-Out - Accuracy: 0.9200
Matriz de Confusión:
 [[50  0  0]
 [ 0 45  5]
 [ 0  7 43]]

Clasificador: 1-NN
Hold Out (70/30) - Accuracy: 0.9778
Matriz de Confusión:
 [[10  0  0]
 [ 0 17  0]
 [ 0  1 17]]
10-Fold Cross-Validation - Accuracy: 0.9600
Matriz de Confusión:
 [[50  0  0]
 [ 0 47  3]
 [ 0  3 47]]
Leave-One-Out - Accuracy: 0.9600
Matriz de Confusión:
 [[50  0  0]
 [ 0 47  3]
 [ 0  3 47]]

----------------------------------------

Dataset: Wine

Clasificador: Euclidean
Hold Out (70/30) - Accuracy: 0.7222
Matriz de Confusión:
 [[12  0  3]
 [ 1 17  4]
 [ 1  6 10]]
10-Fold Cross-Validation - Accuracy: 0.7235
Matriz de Confusión:
 [[49  0  9]
 [ 3 46 18]
 [ 1 16 28]]
Leave-One-Out - Accuracy: 0.7247
Matriz de Confusión:
 [[50  0  9]
 [ 3 49 19]
 [

Este código recorre cada conjunto de datos almacenado en `dataset_data`, imprimiendo primero el nombre del conjunto de datos. Para cada conjunto, aplica dos clasificadores: `euclidean_classifier` y `knn_1_classifier`. Dentro de cada clasificador, se evalúan tres métodos de validación: Hold-Out, 10-Fold Cross-Validation y Leave-One-Out. En la validación Hold-Out, la función `hold_out_validation` divide los datos en un 70% para entrenamiento y 30% para prueba, calculando la exactitud (`accuracy`) y la matriz de confusión (`conf_matrix`). En la validación 10-Fold Cross-Validation, `cross_validation_kfold` divide los datos en 10 pliegues, donde cada pliegue se usa sucesivamente como conjunto de prueba, acumulando la exactitud media y una matriz de confusión global. En Leave-One-Out, `leave_one_out_validation` evalúa cada muestra como conjunto de prueba individual mientras el resto sirve de entrenamiento, generando una exactitud media y una matriz de confusión sumada. Para cada método de validación, imprime la exactitud con cuatro decimales y la matriz de confusión resultante. Al final de cada clasificador, se imprime una línea divisoria para separar los resultados de cada conjunto de datos.

## Pruebas

In [8]:
from sklearn.model_selection import train_test_split, cross_val_score, LeaveOneOut, KFold
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid

# Cargar datasets
datasets = {
    "Iris": load_iris(),
    "Wine": load_wine(),
    "Breast Cancer": load_breast_cancer()
}

# Diccionario para almacenar los clasificadores
classifiers = {
    "Euclidean": NearestCentroid(),
    "1-NN": KNeighborsClassifier(n_neighbors=1)
}

# Función para realizar validación hold-out
def hold_out_validation(X, y, classifier):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)
    return accuracy, conf_matrix

# Función para realizar validación cruzada k-fold
def cross_validation_kfold(X, y, classifier, k=10):
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    accuracies = cross_val_score(classifier, X, y, cv=kf)
    classifier.fit(X, y)
    y_pred = cross_val_score(classifier, X, y, cv=kf, scoring="accuracy")
    # Generamos la matriz de confusión sumando todas las matrices de los pliegues
    conf_matrix_sum = np.zeros((len(np.unique(y)), len(np.unique(y))), dtype=int)
    for train_idx, test_idx in kf.split(X):
        classifier.fit(X[train_idx], y[train_idx])
        y_pred = classifier.predict(X[test_idx])
        conf_matrix_sum += confusion_matrix(y[test_idx], y_pred)
    return np.mean(accuracies), conf_matrix_sum

# Función para realizar leave-one-out cross validation
def leave_one_out_validation(X, y, classifier):
    loo = LeaveOneOut()
    y_true, y_pred = [], []
    for train_idx, test_idx in loo.split(X):
        classifier.fit(X[train_idx], y[train_idx])
        pred = classifier.predict(X[test_idx])
        y_true.extend(y[test_idx])
        y_pred.extend(pred)
    accuracy = accuracy_score(y_true, y_pred)
    conf_matrix = confusion_matrix(y_true, y_pred)
    return accuracy, conf_matrix

# Aplicamos las validaciones a cada dataset y clasificador
for dataset_name, data in datasets.items():
    X, y = data.data, data.target
    print(f"Dataset: {dataset_name}")
    for classifier_name, classifier in classifiers.items():
        print(f"\nClasificador: {classifier_name}")
        
        # Hold-Out
        accuracy, conf_matrix = hold_out_validation(X, y, classifier)
        print(f"Hold Out (70/30) - Accuracy: {accuracy:.4f}")
        print("Matriz de Confusión:\n", conf_matrix)
        
        # 10-Fold Cross-Validation
        accuracy, conf_matrix = cross_validation_kfold(X, y, classifier)
        print(f"10-Fold Cross-Validation - Accuracy: {accuracy:.4f}")
        print("Matriz de Confusión:\n", conf_matrix)
        
        # Leave-One-Out
        accuracy, conf_matrix = leave_one_out_validation(X, y, classifier)
        print(f"Leave-One-Out - Accuracy: {accuracy:.4f}")
        print("Matriz de Confusión:\n", conf_matrix)
    print("\n" + "-"*40 + "\n")


Dataset: Iris

Clasificador: Euclidean
Hold Out (70/30) - Accuracy: 0.9556
Matriz de Confusión:
 [[19  0  0]
 [ 0 11  2]
 [ 0  0 13]]
10-Fold Cross-Validation - Accuracy: 0.9267
Matriz de Confusión:
 [[50  0  0]
 [ 0 45  5]
 [ 0  6 44]]
Leave-One-Out - Accuracy: 0.9200
Matriz de Confusión:
 [[50  0  0]
 [ 0 45  5]
 [ 0  7 43]]

Clasificador: 1-NN
Hold Out (70/30) - Accuracy: 1.0000
Matriz de Confusión:
 [[19  0  0]
 [ 0 13  0]
 [ 0  0 13]]
10-Fold Cross-Validation - Accuracy: 0.9600
Matriz de Confusión:
 [[50  0  0]
 [ 0 47  3]
 [ 0  3 47]]
Leave-One-Out - Accuracy: 0.9600
Matriz de Confusión:
 [[50  0  0]
 [ 0 47  3]
 [ 0  3 47]]

----------------------------------------

Dataset: Wine

Clasificador: Euclidean
Hold Out (70/30) - Accuracy: 0.7593
Matriz de Confusión:
 [[17  0  2]
 [ 0 14  7]
 [ 0  4 10]]
10-Fold Cross-Validation - Accuracy: 0.7248
Matriz de Confusión:
 [[50  0  9]
 [ 3 49 19]
 [ 1 17 30]]
Leave-One-Out - Accuracy: 0.7247
Matriz de Confusión:
 [[50  0  9]
 [ 3 49 19]
 [